## Effect on theta asymmetry if slow gamma frequency region is filtered out
* bandstop filter

In [2]:
import numpy as np
import scipy as sg
import subjects
from plotUtil import Fig
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline
sessions = subjects.Of().ratNday4
for sub,sess in enumerate(sessions):
    eegsrate = sess.recinfo.lfpSrate
    nyq = 0.5 * eegsrate
    maze = sess.epochs.maze
    lfpmaze = sess.recinfo.geteeg(chans=111,timeRange=maze)
    b,a = sg.signal.butter(5,[16/nyq,20/nyq],btype='bandstop')
    yf = sg.signal.filtfilt(b,a,lfpmaze)
    plt.plot(lfpmaze,'k')
    plt.plot(yf,'r')

## Power correlation to identify fundamental frequencies 

In [2]:
import numpy as np
import scipy.signal as sg
import matplotlib.pyplot as plt
import subjects
from plotUtil import Fig

In [7]:
%matplotlib widget
figure = Fig()
fig,gs = figure.draw(num=1,grid=(3,3))
sessions = subjects.Of().ratNday4
for sub,sess in enumerate(sessions):
    eegsrate = sess.recinfo.lfpSrate
    maze =sess.epochs.maze
    speed = sess.position.data['speed']
    channels = sess.recinfo.goodchangrp[3][::2]
    for i, chan in enumerate(channels):
        # lfpmaze = sess.recinfo.geteeg(chans=chan,timeRange=maze)
        f,t,sxx = sess.recinfo.get_specgram(chan=chan,window=2,overlap=1,period=maze)

        f_req_ind = np.where((1<f) & (f<100))[0]
        f_req = f[f_req_ind]
        corr_freq = np.corrcoef(sxx[f_req_ind,:])
        np.fill_diagonal(corr_freq,val=0)

        ax = plt.subplot(gs[i])
        ax.pcolormesh(f_req,f_req,corr_freq,shading='auto',cmap='bwr',vmax=0.3)
        # ax.set_xscale('log')
        # ax.set_yscale('log')
        ax.set_xticks([8,16,30,50,60,90])
        ax.set_yticks([8,16,30,50,60,90])
        ax.set_xlabel('Frequency (Hz)')
        ax.set_ylabel('Frequency (Hz)')
        

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Power power correlations at different frequencies for unit-LFP
* Wanted to see if spike timings also reflect distinct bands in power-power correlations
* First bin the spiketimes into 1ms bins and then from that spkcount timeseries do the power power correlation

### observations
* spike count timeseries has very sharp transitions and a fourier transform of that will have power in all frequencies which is also seen in the figures as a 'continuous band'

In [5]:
import numpy as np
import scipy.signal as sg
import matplotlib.pyplot as plt
import subjects
from plotUtil import Fig

In [8]:
%matplotlib widget
figure = Fig()
fig,gs = figure.draw(num=1,grid=(4,3))
sessions = subjects.Of().ratNday4
for sub,sess in enumerate(sessions):
    eegsrate = sess.recinfo.lfpSrate
    cell_ids= sess.spikes.pyrid
    maze =sess.epochs.maze
    spkcount = sess.spikes.get_spkcounts(cell_ids=cell_ids,period=maze,binsize=0.001)

    for cell,spk in enumerate(spkcount[20:30,:]):
        f,t,sxx = sg.spectrogram(spk,fs=1000,nperseg=2*1000,noverlap=1000) 
        f_req_ind = np.where((1<f) & (f<100))[0]
        f_req = f[f_req_ind]
        corr_freq = np.corrcoef(sxx[f_req_ind,:])
        np.fill_diagonal(corr_freq,val=0)

        ax= plt.subplot(gs[cell])
        ax.pcolormesh(f_req,f_req,corr_freq,shading='auto',cmap='bwr')
        # ax.set_xscale('log')
        # ax.set_yscale('log')
        ax.set_xticks([8,16,30,50,60,90])
        ax.set_yticks([8,16,30,50,60,90])
        ax.set_xlabel('Frequency (Hz)')
        ax.set_ylabel('Frequency (Hz)')
        

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Power power correlation for phase-broken lfp
correlation between powers at different frequencies is used to estimate fundamental frequency. Here we are estimating these fundamental frequency bands by breaking theta into phase-specific bins, which are then concatenated across cycles to generate phase-specific lfp. Interesting comparisons between "highpass filtered lfp (25 Hz)" and breaking raw theta itself. 

In [2]:
import numpy as np
import scipy.signal as sg
import matplotlib.pyplot as plt
import subjects
from plotUtil import Fig
import signal_process
import pandas as pd

In [4]:
%matplotlib qt
figure = Fig()
fig,gs = figure.draw(num=1,grid=(4,2))
sessions = subjects.Of().ratNday4
for sub,sess in enumerate(sessions):
    eegsrate = sess.recinfo.lfpSrate
    maze =sess.epochs.maze
    speed = sess.position.data['speed']
    channels = sess.recinfo.goodchangrp[3][::4]
    for i, chan in enumerate(channels):
        lfpmaze = sess.recinfo.geteeg(chans=chan,timeRange= maze)
        strong_theta = sess.theta.getstrongTheta(lfpmaze)[0]
        slgamma_highpass = signal_process.filter_sig.highpass(strong_theta, cutoff=25)
        gamma_bin, _, angle_centers = sess.theta.phase_specfic_extraction(
            strong_theta, slgamma_highpass, binsize=180, slideby=None
        )
        df = pd.DataFrame()
        f_ = None
        for ii,(lfp, center) in enumerate(zip(gamma_bin, angle_centers)):
            f,t,sxx = sg.spectrogram(lfp,fs=eegsrate,nperseg=2*eegsrate,noverlap=1*eegsrate)
            f_req_ind = np.where((1<f) & (f<100))[0]
            f_req = f[f_req_ind]
            corr_freq = np.corrcoef(sxx[f_req_ind,:])
            np.fill_diagonal(corr_freq,val=0)

            ax = plt.subplot(gs[i,ii])
            ax.pcolormesh(f_req,f_req,corr_freq,shading='gouraud',cmap='coolwarm',vmin=-0.1,vmax=0.1)
            # ax.set_xscale('log')
            # ax.set_yscale('log')
            ax.set_xticks([8,16,30,50,60,90])
            ax.set_yticks([8,16,30,50,60,90])
            ax.set_xlabel('Frequency (Hz)')
            ax.set_ylabel('Frequency (Hz)')


In [13]:
gamma_bin[0]

array([-67.81615977,  94.79806004, 181.58017367, ...,  67.31187149,
        28.82862888, 103.98485404])

## Bicoherence for phase-broken lfp (similar to above)

In [2]:
import numpy as np
import scipy.signal as sg
import matplotlib.pyplot as plt
import subjects
from plotUtil import Fig
import signal_process
import pandas as pd

In [3]:
%matplotlib qt
figure = Fig()
fig,gs = figure.draw(num=1,grid=(4,2))
sessions = subjects.Of().ratNday4
for sub,sess in enumerate(sessions):
    eegsrate = sess.recinfo.lfpSrate
    maze =sess.epochs.maze
    speed = sess.position.data['speed']
    channels = sess.recinfo.goodchangrp[3][::4]
    for i, chan in enumerate(channels):
        lfpmaze = sess.recinfo.geteeg(chans=chan,timeRange= maze)
        strong_theta = sess.theta.getstrongTheta(lfpmaze)[0]
        slgamma_highpass = signal_process.filter_sig.highpass(strong_theta, cutoff=25)
        gamma_bin, _, angle_centers = sess.theta.phase_specfic_extraction(
            strong_theta, strong_theta, binsize=180, slideby=None
        )
        df = pd.DataFrame()
        f_ = None
        for ii,(lfp, center) in enumerate(zip(gamma_bin, angle_centers)):
            f,t,sxx = sg.spectrogram(lfp,fs=eegsrate,nperseg=2*eegsrate,noverlap=1*eegsrate)
            f_req_ind = np.where((1<f) & (f<100))[0]
            f_req = f[f_req_ind]
            corr_freq = np.corrcoef(sxx[f_req_ind,:])
            np.fill_diagonal(corr_freq,val=0)

            ax = plt.subplot(gs[i,ii])
            ax.pcolormesh(f_req,f_req,corr_freq,shading='gouraud',cmap='coolwarm',vmin=-0.1,vmax=0.1)
            # ax.set_xscale('log')
            # ax.set_yscale('log')
            ax.set_xticks([8,16,30,50,60,90])
            ax.set_yticks([8,16,30,50,60,90])
            ax.set_xlabel('Frequency (Hz)')
            ax.set_ylabel('Frequency (Hz)')
